In [23]:
import numpy as np
import pandas as pd
import os
import glob
import json
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline

# import tensorflow as tf

from sklearn import tree
from sklearn import metrics
from sklearn.model_selection import train_test_split


import os
import datetime
from zoneinfo import ZoneInfo
import pandas as pd
import numpy as np
import ffmpeg
import PIL
from PIL import Image
from PIL import ImageOps
# from PIL import Resampling
from matplotlib import cm

In [2]:
!ls

adl001_mat.mp4	       centre1_mat.png	   corner4_mat.png
adl001_mixed.mp4       centre1.mp4	   corner4.mp4
adl001.mp4	       corner1_camera.png  data
adl002_mat.mp4	       corner1_mat.mp4	   decision-tree-classifer.ipynb
adl002_mixed.mp4       corner1_mat.png	   find_odd.ipynb
adl002.mp4	       corner1.mp4	   LICENSE
adl003_mat.mp4	       corner2_camera.png  make_directories.ipynb
adl003_mixed.mp4       corner2_mat.mp4	   mat_camera_align.ipynb
adl003.mp4	       corner2_mat.png	   mat_corners.png
adl004_mat.mp4	       corner2.mp4	   read_mat_camera_data.ipynb
adl004_mixed.mp4       corner3_camera.png  read_timestamps.ipynb
adl004.mp4	       corner3_mat.mp4	   rename_as_timestamp.ipynb
camera_all_points.png  corner3_mat.png	   rename_as_timestamp.py
camera_rescaled.png    corner3.mp4	   subject1.7z
centre1_camera.png     corner4_camera.png
centre1_mat.mp4        corner4_mat.mp4


# Load the data, find "signatures"

"Signature" is number of "lit" pixels, and sum of all lit pixels.

Discard frames with fewer than 10 lit pixels.

In [3]:
def fall_signature(frame, threshold=10):
    return pd.DataFrame({'s': frame[frame > threshold].sum(axis=1), 'n': frame[frame > threshold].count(axis=1)})

The outer loop goes through each file in each directory in the `data` directory. If the file is in a directory that has `Fall` in the name, and the filename ends `csv`, read it into a Pandas dataframe. Drop the columns that aren't the pressure data, then calculate the "signatures" of each frame in that file. Store the signatures, keyed by the filename, in the `signatures_f` dictionary.

In [4]:
signatures_f = {}
for root, dirs, files in os.walk('data/mat_data/'):
    if 'Fall' in root:
        for file in files:
            if dirs == [] and  file[-3:] == 'csv':
                f = os.path.join(root, file)
                fr = pd.read_csv(f)
                fr.drop(['Frame', 'Timestamp', "Range Min (mmHg)", "Range Max (mmHg)"], axis=1, inplace=True)
                sig = fall_signature(fr)
                signatures_f[f] = sig

Let's look at a sample "signature" for a file.

In [33]:
tr = signatures_f['data/mat_data/Blaine/Falls PM/BLFPM06/b6.csv']
tr

,s,n
0,7517.0,141
1,7551.0,141
2,7594.0,146
3,7532.0,141
4,7546.0,144
...,...,...
174,7668.0,145
175,7620.0,142
176,7678.0,144
177,7662.0,141


Each row has the sum of all "lit" pixels and the number of pixels.

We can pick out the row indices of all the rows with more then 143 lit pixels and a total activation of over 1000, and put those indices in a Python list.

In [ ]:
tr.index[(tr.n > 143) & (tr.s > 1000)].to_list()

[2,
 4,
 5,
 6,
 9,
 12,
 13,
 14,
 15,
 16,
 17,
 19,
 22,
 28,
 39,
 42,
 43,
 48,
 49,
 50,
 51,
 52,
 61,
 64,
 65,
 70,
 79,
 84,
 86,
 87,
 90,
 91,
 94,
 95,
 96,
 97,
 98,
 99,
 101,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 113,
 115,
 116,
 117,
 118,
 120,
 121,
 122,
 124,
 125,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 157,
 168,
 169,
 172,
 173,
 174,
 176,
 178]

Let's find the "weird" frames, the red dots in the sea of blue. Note that all the records we're looking at are labelled "falls".

A frame is "weird" if it has fewer than 50 lit pixels, and the average activation is over 70.

We go through all the traces in `signatures_f`, keep only the ones with more than 10 lit pixels, find the average activation, and record the indices in `weirds` of the weird frames. If there are more than 10 weird frames in a single recording, capture them in `weird_frames`.

In [30]:
weird_frames = {}
for f in signatures_f:
    tr = signatures_f[f]
    tr = tr[tr.n > 10]
    tr['a'] = tr.s / tr.n
    weirds = tr.index[(tr.a > 70) & (tr.n < 50)].to_list()
    if len(weirds) >= 10:
        weird_frames[f] = weirds
weird_frames

/tmp/ipykernel_318678/619866155.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tr['a'] = tr.s / tr.n


{'data/mat_data/Lyne/Falls Mat/LMATFALL29/29L.csv': [0,
  1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  20,
  21,
  22,
  23,
  24,
  25,
  26,
  27,
  28,
  33,
  34,
  35,
  36,
  37,
  38,
  40,
  42,
  45,
  46,
  47,
  48,
  49,
  51,
  52,
  53,
  54,
  55,
  57,
  111,
  112,
  283,
  284,
  291,
  292,
  293,
  295,
  296,
  303,
  304,
  308,
  309,
  310,
  314,
  315,
  319,
  320,
  321,
  322,
  323,
  324,
  325,
  326,
  327,
  328,
  329,
  330,
  331,
  332,
  333,
  334,
  335,
  336,
  337,
  338,
  339,
  340,
  341,
  342,
  343,
  344,
  345,
  346,
  347,
  348,
  349,
  350,
  351,
  352,
  353,
  354,
  355],
 'data/mat_data/Lyne/Falls Mat/LMATFALL36/36L.csv': [0,
  1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  20,
  21,
  22,
  23,
  24,
  25,
  26,
  27,
  28,
  29,
  30,
  31,
  32,
  33,
  34,
  35,
  36,
  37,
  38,
  39,
  40,
  41,
  42,


Show the filenames of files with weird frames.

In [32]:
weird_frames.keys()

dict_keys(['data/mat_data/Lyne/Falls Mat/LMATFALL29/29L.csv', 'data/mat_data/Lyne/Falls Mat/LMATFALL36/36L.csv', 'data/mat_data/Clive/09_07 CFalls Pressure mat/CFPM36/c36.csv', 'data/mat_data/Clive/09_07 CFalls Pressure mat/CFPM29/29c.csv', 'data/mat_data/Pablo/Falls PM/PDRPMF36/36p.csv', 'data/mat_data/Pablo/Falls PM/PDRPMF01/1p.csv', 'data/mat_data/Pablo/Falls PM/PDRPMF29/29p.csv', 'data/mat_data/Marion/Falls/Dataset MB1_06 PMF/29/29PM.csv', 'data/mat_data/Marion/Falls/Dataset MB1_06 PMF/36/36PM.csv'])

Let's look at one such track.

Some magic numbers for aligning the mat and camera data, so we can create a composite image of them both.

In [20]:
camera_image_width, camera_image_height = 80, 60
camera_image_rescale = 1.5

mat_image_width, mat_image_height = 27, 64
camera_min_value, camera_max_value = 7886.0, 8171.0

Utility function to parse the datetime information in the mat data.

In [21]:
def mat_date_reader(d):
    try:
        return datetime.datetime.strptime(d, '%Y-%m-%dT%H:%M:%S.%f%z')
    except ValueError:
        return datetime.datetime.strptime(d, '%Y-%m-%dT%H:%M:%S%z')

Function to convert a mat csv file into a video.

In [22]:
def vidwrite_mat(fn, images, framerate=30, vcodec='libx264'):
    if not isinstance(images, np.ndarray):
        images = np.asarray(images)
    n,height,width = images.shape
    width += 1
    process = (
        ffmpeg
            .input('pipe:', format='rawvideo', pix_fmt='rgb24', s=f'{width}x{height}')
            .output(fn, pix_fmt='yuv420p', vcodec=vcodec, r=framerate,
                    vf="drawtext=fontfile=Arial.ttf: text=%{n}: x=0: y=5: fontcolor=white: box=1: boxcolor=0x00000099"
                   )
            .overwrite_output()
            .run_async(pipe_stdin=True)
    )
    for frame in images:
        mod_frame = np.hstack((frame, np.zeros((64, 1), dtype=frame.dtype)))
        process.stdin.write(
            (cm.hot(mod_frame) * 255)[:, : , :-1]
                .astype(np.uint8)
                .tobytes()
        )
    process.stdin.close()
    process.wait()

Read one particular data file.

In [24]:
mframe = pd.read_csv('data/mat_data/Lyne/Falls Mat/LMATFALL29/29L.csv', 
                     converters={'Timestamp': mat_date_reader},
                    index_col='Frame')
mframe.dtypes

Timestamp           datetime64[ns, UTC+01:00]
Range Min (mmHg)                        int64
Range Max (mmHg)                        int64
0                                       int64
1                                       int64
                              ...            
1723                                    int64
1724                                    int64
1725                                    int64
1726                                    int64
1727                                    int64
Length: 1731, dtype: object

Now we're going to turn this into a movie.

Drop the non-image columns.

Convert the dataframe into a Numpy array, convert the datatype to float, normalise the values to the range [0-1], reshape into a 3d array. First dimension is frame number, second is max long axis, third is mat short axis.

In [25]:
mat_frames = mframe.drop(['Timestamp', 'Range Min (mmHg)', 'Range Max (mmHg)'], axis='columns')
mat_frames = (mat_frames.to_numpy().astype(np.float64) / 104).reshape(-1, 64, 27)
mat_frames

array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       ...,

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0.

Experimenting with conversion from that format to something that can be an image. Apply the colour map (`cm`) to the array, multiply by 255, convert to unsigned 8-bit integers (exactly one byte). The PIL library needs this format to convert the array into an image.

In [26]:
((cm.hot(mat_frames[0]) * 255)[... , :-1]
     .astype(np.uint8)
     # .tobytes()
)

array([[[10,  0,  0],
        [10,  0,  0],
        [10,  0,  0],
        ...,
        [10,  0,  0],
        [10,  0,  0],
        [10,  0,  0]],

       [[10,  0,  0],
        [10,  0,  0],
        [10,  0,  0],
        ...,
        [10,  0,  0],
        [10,  0,  0],
        [10,  0,  0]],

       [[10,  0,  0],
        [10,  0,  0],
        [10,  0,  0],
        ...,
        [10,  0,  0],
        [10,  0,  0],
        [10,  0,  0]],

       ...,

       [[10,  0,  0],
        [10,  0,  0],
        [10,  0,  0],
        ...,
        [10,  0,  0],
        [10,  0,  0],
        [10,  0,  0]],

       [[10,  0,  0],
        [10,  0,  0],
        [10,  0,  0],
        ...,
        [10,  0,  0],
        [10,  0,  0],
        [10,  0,  0]],

       [[10,  0,  0],
        [10,  0,  0],
        [10,  0,  0],
        ...,
        [10,  0,  0],
        [10,  0,  0],
        [10,  0,  0]]], dtype=uint8)

Call the function above to write the mat data to a new video file.

In [28]:
vidwrite_mat(f'lyn_fall_29_mat.mp4', mat_frames)

ffmpeg version 7.0 Copyright (c) 2000-2024 the FFmpeg developers
  built with gcc 12.3.0 (conda-forge gcc 12.3.0-7)
  configuration: --prefix=/home/conda/feedstock_root/build_artifacts/ffmpeg_1716144998863/_h_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_plac --cc=/home/conda/feedstock_root/build_artifacts/ffmpeg_1716144998863/_build_env/bin/x86_64-conda-linux-gnu-cc --cxx=/home/conda/feedstock_root/build_artifacts/ffmpeg_1716144998863/_build_env/bin/x86_64-conda-linux-gnu-c++ --nm=/home/conda/feedstock_root/build_artifacts/ffmpeg_1716144998863/_build_env/bin/x86_64-conda-linux-gnu-nm --ar=/home/conda/feedstock_root/build_artifacts/ffmpeg_1716144998863/_build_env/bin/x86_64-conda-linux-gnu-ar --disable-doc --disable-openssl --enable-demuxer=dash --enable-hardcoded-tables --enable-libfreetype --enable-libharfbuzz --enable-libfontconfig --enable-libope